In [ ]:
import pandas as pd
from long_covid.colors import flatuicolors
from long_covid import styling
from matplotlib import pyplot as plt
from matplotlib.colors import to_rgba

In [ ]:
# Global variables for styling
X_LABEL = 'Weeks since PCR-test'
Y_LABELS = ['Change in RHR [bpm]', 'Change in daily steps', 'Change in daily sleep\nduration [minutes]']
    
# Globally accessible input data
COHORTS = pd.read_feather('../data/03_derived/user_cohorts.feather')
WEEKLY_DEVIATIONS = pd.read_feather('../data/03_derived/weekly_vital_deviations_per_user.feather')

In [ ]:
f, axarr = plt.subplots(3, 2, sharex='col', sharey='row', figsize=(6, 6))

colors = [flatuicolors.pomegranate, flatuicolors.amethyst]

for i, cohort in enumerate(['unvaccinated', 'vaccinated']):
    users = COHORTS[COHORTS[cohort]].user_id.values
    c = colors[i]
    
    for j, vital in enumerate([65, 9, 43]):
        ax = axarr[j, i]
        data = WEEKLY_DEVIATIONS[WEEKLY_DEVIATIONS.userid.isin(users) & (WEEKLY_DEVIATIONS.vitalid == vital)]
        
        if cohort == 'unvaccinated':
            data = data[(data.weeks_since_test >= -3) & (data.weeks_since_test <= 15)]
             
        if cohort == 'vaccinated':
            data = data[(data.weeks_since_test >= -3) & (data.weeks_since_test <= 12)]
        
        average = data.groupby('weeks_since_test').vital_change.mean()
        
        if cohort == 'unvaccinated':
            weeks = range(-3, 15)
        else:
            weeks = range(-3, 13)
        
        for week in weeks:
            bplot = ax.boxplot(
                data[data.weeks_since_test == week].vital_change.values, 
                positions=[week],
                patch_artist=True,
                widths=0.75,
                flierprops={'markeredgecolor': c, 'markersize': 2, 'alpha': 0.5},
                medianprops={'c': c, 'lw': 1.5},
            )
            
            bplot['boxes'][0].set_facecolor(to_rgba(c, 0.2))
            bplot['boxes'][0].set_edgecolor('k')
            
        styling.hide_and_move_axis(ax)
                
axarr[0, 0].set_xticks(range(-2, 15, 2))
axarr[0, 1].set_xticks(range(-2, 13, 2))

axarr[0, 0].set_xticklabels(range(-2, 15, 2))
axarr[0, 1].set_xticklabels(range(-2, 13, 2))

for ax in axarr[-1, :]:
    ax.set_xlabel(X_LABEL)
    
for ax, label in zip(axarr[:, 0], Y_LABELS):
    ax.set_ylabel(label)

axarr[0, 0].set_title('Unvaccinated')
axarr[0, 1].set_title('Vaccinated')

plt.tight_layout()
plt.savefig('../output/si_figure8_dot_plot_vital_changes.pdf')